<a href="https://colab.research.google.com/github/parhambz/corona_diag_tensorflow/blob/main/corona_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Diagnosing coronavirus using tensorflow




## Project description


It is believed by some in the future computers can work as a helping hand for the doctors especially in diagnosing diseases.Since corona virus broke out the over population of the hospitals have been a critical problem.A large number of people visit the hospitals only to be checked whether they are infected by the virus or not while there is no need to go to hospital in person.And with the advent of artificial intelligence there are many ways to separate different types of people only using the computers.In this project we try to train a model in order to facilitate the process of diagnosing coronavirus without any doctor being involved.


---


The project is developed using python programming language and Tensorflow which is a library written in python.
The developed code will be explained:

## Code

**Selecting the proper version of tensorflow**



In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


**importing essential libraries**

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import torch
import pandas as pd
import numpy as np

**defining the headings of the input data**

The given table provides information on 8 decisive factor of recodgnizing corona which are :


1.   Cough
2.   Fever
3.   Sore throat
4.   Shortness of breath
5.   Headache
6.   Corona test result
7.   Age
8.   Gender




In [ ]:
CSV_COLUMN_NAMES = ['cough', 'fever', 'sore_t', 'shortness_breath', 'headache','corona_res','age_more60','gender','test_ind']
cov_resault = ['positive','negative']
# Lets define some constants to help us later on


**Loading the input data**

In this section provided data for train is loaded from my privately owned google drive

In [ ]:

train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/corona_tested_individuals_ver_0083.english.csv", names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


**Preparing the given data**


*   First null values in the input data are replaced with empty values ( ' ' ).




In [ ]:
train.fillna('', inplace=True)




*   In the next stage the given data table is partitioned into test data and train data by mean of the function defined below.



In [ ]:
def get_dataset_partitions_pd(df, train_split=0.8, val_split=0.1, test_split=0.1):
    assert (train_split + test_split + val_split) == 1
    
    # Only allows for equal validation and test splits
    assert val_split == test_split 

    # Specify seed to always have the same split distribution between runs
    df_sample = df.sample(frac=1, random_state=12)
    indices_or_sections = [int(train_split * len(df)), int((1 - val_split - test_split) * len(df))]
    
    train_ds, val_ds, test_ds = np.split(df_sample, indices_or_sections)
    
    return train_ds, val_ds, test_ds

In [ ]:
(train,val,test)=get_dataset_partitions_pd(train, train_split=0.8, val_split=0.1, test_split=0.1)



*   Following the preparing process in this stage the labels are extracted from the train and test data and then they are encoded (0 for negative and 1 for positive) in order to use them in training and evaluation process.


In [ ]:
train_ys = train.pop('corona_res')
train_y=[]
for r in train_ys:
  if (r=="positive"):
    train_y.append(1)
  else:
    train_y.append(0)
train_y = np.array(train_y)


In [ ]:
test_ys = test.pop('corona_res')
test_y=[]
for r in test_ys:
  if (r=="positive"):
    test_y.append(1)
  else:
    test_y.append(0)
test_y = np.array(test_y)



*   This stage defines categorical columns in our data in order to encode them to integer values 


In [ ]:
CATEGORICAL_COLUMNS = [ 'gender', 'test_ind','age_more60','cough', 'fever','sore_t','shortness_breath','headache']
#NUMERIC_COLUMNS=['cough', 'fever','sore_t','shortness_breath','headache']

feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = train[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary)))

#for feature_name in NUMERIC_COLUMNS:
#  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.boolean_mask))

**Defining the input function**

The defined function below is responsible of feeding the model by giving the input data to the model in a desired way.

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)


**Creating the model**

In this section we create a deep neural network model to be trained to be able to classify any data.

In [ ]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.q
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpxtqqqm6b', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


**Training**

The training starts in this section and the data is given by the function which was defined before.

In [ ]:

classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000)
# We include a lambda to avoid creating an inner function previously

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpxtqqqm6b/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.8714495, step = 0
INFO:tensorflow:global_step/sec: 138.782
INFO:tensorflow:loss = 0.58082414, step = 100 (0.722 sec)
INFO:tensorflow:global_step/sec: 384.394
INFO:tensorflow:loss = 0.531592, step = 200 (0.264 sec)
INFO:tensorflow:global_step/sec

**Evaluation code**

At the end it is important to measure the accuracy of the trained model.In other words we use the separated data which was considered as a test data to calculate the accuracy and how successful our model is in classifying the infected persons from those who are not infected.

In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-05-14T13:53:37
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpxtqqqm6b/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 17.96255s
INFO:tensorflow:Finished evaluation at 2022-05-14-13:53:55
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.91906404, average_loss = 0.24883823, global_step = 5000, loss = 0.24884002
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmpxtqqqm6b/model.ckpt-5000

Test set accuracy: 0.919



## Evaluation

From the result it can be claimed developing the model was almost successful as the evaluation result shows the accuracy of about %92.